## Load images and packages

In [1]:
#The following code snip-it reads any file from the internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
from skimage import exposure #histogram equalization
import colorsys #To convert to rbg to hsv color space
import matplotlib.colors as colors
import os #For getting list of files
from scipy import ndimage #For performing erosion and dilation

#Get list of photo names
fileNames = os.listdir(path='.\cornPhotos')
print(fileNames)
os.chdir(path='.\cornPhotos')

#Define thresholds for isolating tray in photos
hmin = -0.01
hmax = 1.01
smin = 0.05
smax = 1.01
vmin = -0.01
vmax = 1.01

#Function for calculating run lengths in a binary array
#function is from: https://stackoverflow.com/questions/1066758/find-length-of-sequences-of-identical-values-in-a-numpy-array-run-length-encodi
def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

#Loop over photos and crop them
for fileName in fileNames:
    #Load picture
    im = imread(fileName)
    
    #Histogram equalization
    im2 = exposure.equalize_hist(im)
    
    #convert from rgb to hsv color space, pull out matrices
    hsv = colors.rgb_to_hsv(im2)
    h = hsv[:,:,0]; #hue matrix
    s = hsv[:,:,1]; #saturation matrix
    v = hsv[:,:,2]; #value matrix (i.e. brightness)
    
    #Convert to binary image based on thresholds
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    b_img = (b_img & 
        (s > smin) & (s < smax) & 
        (v > vmin) & (v < vmax));
    
    #Clean up binary image with erosion and dilation
    b2 = ndimage.binary_erosion(b_img, iterations = 10)
    b3 = ndimage.binary_dilation(b2, iterations = 20)
    
    #Label objects in binary image
    lab, num_features = ndimage.measurements.label(b3)
    
    #Sum togther rows and columns of binary array to determine which pixels represent the tray (labeled as object 1) 
    a1 = np.sum(lab==1,axis=1)
    a0 = np.sum(lab==1,axis=0)
    
    #Convert binary arrays to logical arrays. Now just need to find longest run of False elements in each array
    al0 = a0 < max(a0)
    al1 = a1 < max(a1)
    
    #Calculate run lengths
    runLengths0 = rle(al0)
    runLengths1 = rle(al1)
    
    #Focus on columns
    ##Find index of where longest run begins
    runs0 = runLengths0[0]
    positions0 =  runLengths0[1]
    maxRun0 = max(runs0)
    result = np.where(runs0 == maxRun0)
    ##Calculate where longest run ends
    index = np.asarray(result)
    startCol = positions0[index].tolist()[0][0]
    endCol = startCol + maxRun0
    
    #Focus on rows
    ##Find index of where longest run begins
    runs1 = runLengths1[0]
    positions1 =  runLengths1[1]
    maxRun1 = max(runs1)
    result = np.where(runs1 == maxRun1)
    ##Calculate where longest run ends
    index = np.asarray(result)
    startRow = positions1[index].tolist()[0][0]
    endRow = startRow + maxRun1
    
    #Crop and save image
    im3 = im2[startRow:endRow,startCol:endCol]
    imsave("cropped_" + fileName, im3)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '.\\cornPhotos'

# GRAVEYARD

## Histogram equalization

In [ ]:
# Code snip-it to do do histogram equalization on an RGB image with R, B and G in the same histogram
im2 = exposure.equalize_hist(im)
plt.imshow(im2)

## Which values are most useful for thresholding?

In [ ]:
import colorsys
import matplotlib.colors as colors

hsv = colors.rgb_to_hsv(im2)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(20,5))
ax1.imshow(hsv[:,:,0], cmap='hsv')
ax1.set_title('Hue - Color')
ax1.set_axis_off()

ax2.imshow(hsv[:,:,1],cmap='gray',vmin=0, vmax=1)
ax2.set_title('Saturation - Amount of Color')
ax2.set_axis_off()

ax3.imshow(hsv[:,:,2],cmap='gray')
ax3.set_title('Value - Brightness')
ax3.set_axis_off()

ax4.imshow(im)
ax4.set_axis_off()

## Determine proper thresholds

In [ ]:
from ipywidgets import interactive,fixed

def hsv_color_threshold(im, hmin=-0.01,hmax=1.01, smin=-0.01,smax=1.01,vmin=-1,vmax=256):
    # Pull out the red, gree and blue matrixes
    hsv = colors.rgb_to_hsv(im)
    h = hsv[:,:,0];
    s = hsv[:,:,1];
    v = hsv[:,:,2];
    
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    
    
    b_img = (b_img & 
         (s > smin) & (s < smax) & 
         (v > vmin) & (v < vmax));
    
    f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,5))
    ax1.imshow(im)
    ax1.set_axis_off()
    
    ax2.imshow(b_img,cmap='gray', vmin=0, vmax=1)
    ax2.set_title('Value - Brightness')
    ax2.set_axis_off()

    plt.show()
    return b_img*1;

w = interactive(hsv_color_threshold, im=fixed(im2),
         hmin=(-0.01,1.01,0.01), hmax=(-0.01,1.01,0.01), 
         smin=(-0.01,1.01,0.01), smax=(-0.01,1.01,0.01), 
         vmin=(-1,256), vmax=(-1,256),__manual=True);
w

## Convert from rgb to hsv

In [ ]:
hsv = colors.rgb_to_hsv(im2)

#Pull out matrices
h = hsv[:,:,0]; #hue matrix
s = hsv[:,:,1]; #saturation matrix
v = hsv[:,:,2]; #value matrix (i.e. brightness)

## Apply thresholds

In [ ]:
hmin = -0.01
hmax = 1.01
smin = 0.05
smax = 1.01
vmin = -0.01
vmax = 1.01

# trick because the color space wraps
if hmin > hmax:
    b_img = (h > hmin) | (h < hmax)
else:
    b_img = (h > hmin) & (h < hmax);
    
    
b_img = (b_img & 
    (s > smin) & (s < smax) & 
    (v > vmin) & (v < vmax));

b_img
plt.imshow(b_img, cmap='gray', vmin=0, vmax=1)

## Apply erosion and dilation

In [ ]:
from scipy import ndimage
after_erosion = ndimage.binary_erosion(b_img, iterations=3)
plt.imshow(after_erosion, cmap='gray', vmin=0, vmax=1)

In [ ]:
after_dilation = ndimage.binary_dilation(b_img, iterations=3)
plt.imshow(after_dilation, cmap='gray', vmin=0, vmax=1)

In [ ]:
b2 = ndimage.binary_erosion(b_img, iterations = 10)
b3 = ndimage.binary_dilation(b2, iterations = 20)
plt.imshow(b3, cmap='gray', vmin=0, vmax=1)

## Crop original photo based on tray

In [ ]:
lab, num_features = ndimage.measurements.label(b3)
plt.imshow(lab==1, cmap='jet')

In [ ]:
a1 = np.sum(lab==1,axis=1)
plt.plot(a1)

In [ ]:
a0 = np.sum(lab==1,axis=0)
plt.plot(a0)

In [ ]:
al0 = a0 < max(a0)
al1 = a1 < max(a1)

In [ ]:
#function is from: https://stackoverflow.com/questions/1066758/find-length-of-sequences-of-identical-values-in-a-numpy-array-run-length-encodi
def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

In [ ]:
runLengths0 = rle(al0)
runLengths1 = rle(al1)

#Result is weird tuple that I can't extract number from
runs0 = runLengths0[0]
positions0 =  runLengths0[1]
maxRun0 = max(runs0)
result = np.where(runs0 == maxRun0)

index = np.asarray(result)
startCol = positions0[index].tolist()[0][0]
endCol = startCol + maxRun0
print(startCol)
print(endCol)

#
runs1 = runLengths1[0]
positions1 =  runLengths1[1]
maxRun1 = max(runs1)
result = np.where(runs1 == maxRun1)

index = np.asarray(result)
startRow = positions1[index].tolist()[0][0]
endRow = startRow + maxRun1
print(startRow)
print(endRow)

In [ ]:
im3 = im2[startRow:endRow,startCol:endCol]
plt.imshow(im3)

In [ ]:
imsave('cropped.jpg', im3)